# Data Process

In [1]:
import numpy as np
import pandas as pd
import csv
import random

### Reading data

In [2]:
!gdown 1uEoAqKmm24QZWAZFxGAl2P5Em6MRHYc0
!gdown 1HrpJqWZiXGTNBOEsCM91VsEOZuguaO82
!gdown 1Jq51h4ON-_ikPrqHiUwp_JMDcv0wEDXE
!gdown 1-VjFUQ7rfNXwir20KHktVRksiw-Kpgui
!gdown 1gBEbTdDCpKYGV8IuX7r4eMT1G31ErcIf
!gdown 1WjR_aeW-E3JNilYtVMyAhM_HvwyZuxvh
!gdown 1VzR_ZtcOURSSrK95lrVUUAJIqHD7THSO
!gdown 1HAjCVzQvF_3P8rsx9lmUWEo_bg4s6Rli
!gdown 1ngHte_RuGvuNCWPAUchu_T89l9VeFYya

Downloading...
From: https://drive.google.com/uc?id=1uEoAqKmm24QZWAZFxGAl2P5Em6MRHYc0
To: /content/public_train_x_remit1_full_hashed.csv
100% 1.35M/1.35M [00:00<00:00, 130MB/s]
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1HrpJqWZiXGTNBOEsCM91VsEOZuguaO82 

Downloading...
From: https://drive.google.com/uc?id=1Jq51h4ON-_ikPrqHiUwp_JMDcv0wEDXE
To: /content/public_train_x_cdtx0001_full_hashed.csv
100% 85.4M/85.4M [00:01<00:00, 45.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-VjFUQ7rfNXwir20KHktVRksiw-Kpgui
To: /content/public_train_x_custinfo_full_hashed.csv
100% 2.29M/2.29M [00:00<00:00, 154MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gBEbTdDCpKYGV8IuX7r4eMT1G31ErcIf
To: /content/public_train_x_ccba_full_hashed.csv
100% 6.78M/6.78M 

In [3]:
data_file_path          = './'
data_file_name          = 'public_train_x_{fname}_full_hashed.csv'
information_file_name   = 'custinfo'
credit_file_name        = 'ccba'
consumption_file_name   = 'cdtx0001'
remit_file_name         = 'remit1'
trading_file_name       = 'dp'

In [4]:
information_data    = pd.read_csv(data_file_path + data_file_name.format(fname = information_file_name))
credit_data         = pd.read_csv(data_file_path + data_file_name.format(fname = credit_file_name), index_col='cust_id')
consumption_data    = pd.read_csv(data_file_path + data_file_name.format(fname = consumption_file_name), index_col = 'cust_id')
remit_data          = pd.read_csv(data_file_path + data_file_name.format(fname = remit_file_name), index_col = 'cust_id')
# trading_data        = pd.read_csv(data_file_path + data_file_name.format(fname = trading_file_name), index_col = 'cust_id')
train_x = pd.read_csv(data_file_path + 'train_x_alert_date.csv')
train_y = pd.read_csv(data_file_path + 'train_y_answer.csv')
public_x = pd.read_csv(data_file_path + 'public_x_alert_date.csv')
public_y = pd.read_csv(data_file_path + '24_ESun_public_y_answer.csv')

### Load training data x

In [5]:
train_x_alert_key = []
train_x_alert_date = []
train_x_cust_id = []
train_x_size = len(train_x)
for i in range(train_x_size):
    train_x_alert_key.append(train_x['alert_key'][i])
    train_x_alert_date.append(train_x['date'][i])
    train_x_cust_id.append(information_data['cust_id'][i + 1845])
train_x_alert_date = [x for _, x in sorted(zip(train_x_alert_key, train_x_alert_date))]
train_x_alert_key.sort()


### Load training data y

In [8]:
train_y_alert_key = []
train_y_sar_flag = []
train_y_size = len(train_y)
for i in range(train_y_size):
    train_y_alert_key.append(train_y['alert_key'][i])
    train_y_sar_flag.append(train_y['sar_flag'][i])
train_y_sar_flag = [y for _, y in sorted(zip(train_y_alert_key, train_y_sar_flag))]
train_y_alert_key.sort()

pd.DataFrame(train_y_sar_flag).to_csv('train_y.csv')

### $V_{info}$

In [9]:
V_info = []
for i in range(train_x_size):
    risk_rank       = information_data['risk_rank'][i + 1845]
    occupation_code = information_data['occupation_code'][i + 1845]
    total_asset     = information_data['total_asset'][i + 1845]
    AGE             = information_data['AGE'][i + 1845]
    V_info.append([risk_rank, occupation_code, total_asset, AGE])


pd.DataFrame(V_info).to_csv('V_info.csv')

### $V_{cred}$

In [10]:
import random
V_cred = []
month = [0, 30, 61, 91, 122, 153, 183, 214, 244, 275, 306, 334, 365]
for i in range(train_x_size):
    V_cred_i = []
    if train_x_cust_id[i] not in credit_data.index:
        V_cred_i = [0] * 8 * 13
    # search_cust = credit_data[credit_data['cust_id'] == train_x_cust_id[i]]
    else:
        search_cust = credit_data.loc[[train_x_cust_id[i]]]
        which_month = 13
        for j in range (len(month)):
            if (month[j] > train_x_alert_date[i]):
                which_month = j
                break
        for j in range (which_month-13, which_month):
            m = 0
            if (j >= 0):
                m = month[j]
            search_month = search_cust[search_cust['byymm'] == m]
            if len(search_month.to_numpy()) == 0:
                V_cred_i.extend([0, 0, 0, 0, 0, 0, 0, 0])
            else:
                lupay = search_month['lupay'].to_numpy()[0]
                cycam = search_month['cycam'].to_numpy()[0]
                usgam = search_month['usgam'].to_numpy()[0]
                clamt = search_month['clamt'].to_numpy()[0]
                csamt = search_month['csamt'].to_numpy()[0]
                inamt = search_month['inamt'].to_numpy()[0]
                cucsm = search_month['cucsm'].to_numpy()[0]
                cucah = search_month['cucah'].to_numpy()[0]
                V_cred_i.extend([lupay, cycam, usgam, clamt, csamt, inamt, cucsm, cucah])
        if (random.randint(1,100) == 56):
            print(i, train_x_size, V_cred_i)
    assert (len(V_cred_i) == 104)
    V_cred.append(V_cred_i)

pd.DataFrame(V_cred).to_csv('V_cred.csv')

48 23906 [71681.0, 201968.0, 22696.0, 0.0, 0.0, 4455.0, 118443.0, 0.0, 71681.0, 201968.0, 22696.0, 0.0, 0.0, 4455.0, 118443.0, 0.0, 71681.0, 201968.0, 22696.0, 0.0, 0.0, 4455.0, 118443.0, 0.0, 71681.0, 201968.0, 22696.0, 0.0, 0.0, 4455.0, 118443.0, 0.0, 71681.0, 201968.0, 22696.0, 0.0, 0.0, 4455.0, 118443.0, 0.0, 71681.0, 201968.0, 22696.0, 0.0, 0.0, 4455.0, 118443.0, 0.0, 71681.0, 201968.0, 22696.0, 0.0, 0.0, 4455.0, 118443.0, 0.0, 71681.0, 201968.0, 22696.0, 0.0, 0.0, 4455.0, 118443.0, 0.0, 71681.0, 201968.0, 22696.0, 0.0, 0.0, 4455.0, 118443.0, 0.0, 71681.0, 201968.0, 22696.0, 0.0, 0.0, 4455.0, 118443.0, 0.0, 71681.0, 201968.0, 22696.0, 0.0, 0.0, 4455.0, 118443.0, 0.0, 71681.0, 201968.0, 22696.0, 0.0, 0.0, 4455.0, 118443.0, 0.0, 71681.0, 201968.0, 22696.0, 0.0, 0.0, 4455.0, 118443.0, 0.0]
302 23906 [0.0, 412584.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 412584.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 412584.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 412584.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4

### $V_{cons}$

In [11]:
V_cons = []
month = [0, 30, 61, 91, 122, 153, 183, 214, 244, 275, 306, 334, 365]
for i in range(train_x_size):
    V_cons_i = []
    if train_x_cust_id[i] not in consumption_data.index:
        V_cons_i = [0, 130, 47, 0] * 13
    else:
        search_cust = consumption_data.loc[[train_x_cust_id[i]]]
        country = 130
        cur_type = 47
        if len(search_cust.to_numpy()) != 0:
            country = search_cust['country'].to_numpy()[0]
            cur_type = search_cust['cur_type'].to_numpy()[0]

        which_month = 13
        for j in range (len(month)):
            if (month[j] >= train_x_alert_date[i]):
                which_month = j
                break
        for j in range (which_month-13, which_month):
            down = month[0]
            up = month[1]
            if (j > 0):
                up = month[j]
                down = month[j-1]
            search_month = search_cust[(search_cust['date'] >= down) & (search_cust['date'] < up)]

            n = len(search_month.to_numpy())
            amt = sum(search_month['amt'].to_numpy())
            V_cons_i.extend([n, country, cur_type, amt])
        if (random.randint(1,100) == 56):
            print(i, train_x_size, V_cons_i)
    assert (len(V_cons_i) == 52)
    V_cons.append(V_cons_i)

pd.DataFrame(V_cons).to_csv('V_cons.csv')

81 23906 [5, 130, 47, 22165.0, 5, 130, 47, 22165.0, 5, 130, 47, 22165.0, 5, 130, 47, 22165.0, 5, 130, 47, 22165.0, 5, 130, 47, 22165.0, 5, 130, 47, 22165.0, 5, 130, 47, 22165.0, 5, 130, 47, 22165.0, 5, 130, 47, 22165.0, 5, 130, 47, 22165.0, 5, 130, 47, 22165.0, 5, 130, 47, 22165.0]
481 23906 [33, 130, 47, 17734.0, 33, 130, 47, 17734.0, 33, 130, 47, 17734.0, 33, 130, 47, 17734.0, 33, 130, 47, 17734.0, 33, 130, 47, 17734.0, 33, 130, 47, 17734.0, 33, 130, 47, 17734.0, 33, 130, 47, 17734.0, 33, 130, 47, 17734.0, 33, 130, 47, 17734.0, 33, 130, 47, 17734.0, 33, 130, 47, 17734.0]
1570 23906 [3, 130, 47, 1414.0, 3, 130, 47, 1414.0, 3, 130, 47, 1414.0, 3, 130, 47, 1414.0, 3, 130, 47, 1414.0, 3, 130, 47, 1414.0, 3, 130, 47, 1414.0, 3, 130, 47, 1414.0, 3, 130, 47, 1414.0, 3, 130, 47, 1414.0, 3, 130, 47, 1414.0, 3, 130, 47, 1414.0, 3, 130, 47, 1414.0]
1633 23906 [86, 130, 47, 1934688.0, 86, 130, 47, 1934688.0, 86, 130, 47, 1934688.0, 86, 130, 47, 1934688.0, 86, 130, 47, 1934688.0, 86, 130, 47, 193

### $V_{remit}$

In [16]:
V_remit = []
month = [0, 30, 61, 91, 122, 153, 183, 214, 244, 275, 306, 334, 365]
for i in range(train_x_size):
    V_remit_i = []
    if train_x_cust_id[i] not in remit_data.index:
        V_remit_i = [0] * 3 * 13
    else:
        search_cust = remit_data.loc[[train_x_cust_id[i]]]
        trans_no = 0
        if len(search_cust.to_numpy()) != 0:
            trans_no = search_cust['trans_no'].to_numpy()[0]
        which_month = 13
        for j in range (len(month)):
            if (month[j] >= train_x_alert_date[i]):
                which_month = j
                break

        for j in range (which_month-13, which_month):
            down = month[0]
            up = month[1]
            if (j > 0):
                up = month[j]
                down = month[j-1]
            search_day = search_cust[(search_cust['trans_date'] >= down) & (search_cust['trans_date'] < up)]
            n = len(search_day.to_numpy())
            trade_amount_usd = sum(search_day['trade_amount_usd'].to_numpy())
            V_remit_i.extend([n, trans_no, trade_amount_usd])
        if (random.randint(1,100) == 41):
            print(i, train_x_size, V_remit_i)
    assert (len(V_remit_i) == 39)
    V_remit.append(V_remit_i)

pd.DataFrame(V_remit).to_csv('V_remit.csv')

136 23906 [6, 4, 1131995.0, 6, 4, 1131995.0, 6, 4, 1131995.0, 6, 4, 1131995.0, 6, 4, 1131995.0, 6, 4, 1131995.0, 6, 4, 1131995.0, 6, 4, 1131995.0, 6, 4, 1131995.0, 6, 4, 1131995.0, 6, 4, 1131995.0, 6, 4, 1131995.0, 6, 4, 1131995.0]
2390 23906 [7, 0, 83796.0, 7, 0, 83796.0, 7, 0, 83796.0, 7, 0, 83796.0, 7, 0, 83796.0, 7, 0, 83796.0, 7, 0, 83796.0, 7, 0, 83796.0, 7, 0, 83796.0, 7, 0, 83796.0, 7, 0, 83796.0, 7, 0, 83796.0, 7, 0, 83796.0]
3989 23906 [14, 0, 765791.0, 14, 0, 765791.0, 14, 0, 765791.0, 14, 0, 765791.0, 14, 0, 765791.0, 14, 0, 765791.0, 14, 0, 765791.0, 14, 0, 765791.0, 14, 0, 765791.0, 14, 0, 765791.0, 14, 0, 765791.0, 14, 0, 765791.0, 14, 0, 765791.0]
5116 23906 [0, 3, 0, 0, 3, 0, 0, 3, 0, 0, 3, 0, 0, 3, 0, 0, 3, 0, 0, 3, 0, 0, 3, 0, 0, 3, 0, 0, 3, 0, 0, 3, 0, 0, 3, 0, 0, 3, 0]
5339 23906 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 205424.0]
5389 23906 [2, 3, 71664.0, 2, 3, 71664.0, 2, 3, 71664.0, 2, 3, 

### $V_{trade}$

In [20]:
V_trade = []
month = [0, 30, 61, 91, 122, 153, 183, 214, 244, 275, 306, 334, 365]
trading_data = pd.read_csv(data_file_path + data_file_name.format(fname = trading_file_name), index_col = 'cust_id')

for i in range(train_x_size):
    V_trade_i = []
    if train_x_cust_id[i] not in trading_data.index:
        V_trade_i = [0, 2, 0, 1.0, 0, 0, 0] * 13
    else:
        search_cust = trading_data.loc[[train_x_cust_id[i]]]
        tx_type = 2
        exchg_rate = 1.0
        lin_gui = 0
        cross_bank = 0
        ATM = 0
        if len(search_cust.to_numpy()) != 0:
            tx_type = search_cust['tx_type'].to_numpy()[0]
            exchg_rate = search_cust['exchg_rate'].to_numpy()[0]
            if ((tx_type == 1) and (search_cust['info_asset_code'].to_numpy()[0] == 12)):
                lin_gui = 1
            cross_bank = search_cust['cross_bank'].to_numpy()[0]
            ATM = search_cust['ATM'].to_numpy()[0]

        which_month = 13
        for j in range (len(month)):
            if (month[j] >= train_x_alert_date[i]):
                which_month = j
                break

        for j in range (which_month-13, which_month):
            down = month[0]
            up = month[1]
            if (j > 0):
                up = month[j]
                down = month[j-1]
            search_day = search_cust[(search_cust['tx_date'] >= down) & (search_cust['tx_date'] < up)]

            n = len(search_day.to_numpy())
            tx_amt = sum(search_day['tx_amt'].to_numpy())
            V_trade_i.extend([n, tx_type, tx_amt, exchg_rate, lin_gui, cross_bank, ATM])
        if (random.randint(1,100) == 98):
            print(i, train_x_size, V_trade_i)
    assert (len(V_trade_i) == 91)
    V_trade.append(V_trade_i)
  
pd.DataFrame(V_trade).to_csv('V_trade.csv')

20 23906 [7, 2, 568419.0, 1.0, 0, 0, 0, 7, 2, 568419.0, 1.0, 0, 0, 0, 7, 2, 568419.0, 1.0, 0, 0, 0, 7, 2, 568419.0, 1.0, 0, 0, 0, 7, 2, 568419.0, 1.0, 0, 0, 0, 7, 2, 568419.0, 1.0, 0, 0, 0, 7, 2, 568419.0, 1.0, 0, 0, 0, 7, 2, 568419.0, 1.0, 0, 0, 0, 7, 2, 568419.0, 1.0, 0, 0, 0, 7, 2, 568419.0, 1.0, 0, 0, 0, 7, 2, 568419.0, 1.0, 0, 0, 0, 7, 2, 568419.0, 1.0, 0, 0, 0, 7, 2, 568419.0, 1.0, 0, 0, 0]
259 23906 [36, 2, 5113503.0, 1.0, 0, 0, 0, 36, 2, 5113503.0, 1.0, 0, 0, 0, 36, 2, 5113503.0, 1.0, 0, 0, 0, 36, 2, 5113503.0, 1.0, 0, 0, 0, 36, 2, 5113503.0, 1.0, 0, 0, 0, 36, 2, 5113503.0, 1.0, 0, 0, 0, 36, 2, 5113503.0, 1.0, 0, 0, 0, 36, 2, 5113503.0, 1.0, 0, 0, 0, 36, 2, 5113503.0, 1.0, 0, 0, 0, 36, 2, 5113503.0, 1.0, 0, 0, 0, 36, 2, 5113503.0, 1.0, 0, 0, 0, 36, 2, 5113503.0, 1.0, 0, 0, 0, 36, 2, 5113503.0, 1.0, 0, 0, 0]
486 23906 [90, 1, 76741195.0, 1.0, 0, 0, 1, 90, 1, 76741195.0, 1.0, 0, 0, 1, 90, 1, 76741195.0, 1.0, 0, 0, 1, 90, 1, 76741195.0, 1.0, 0, 0, 1, 90, 1, 76741195.0, 1.0, 0, 0, 

### Store processed data

### Public data

In [24]:
public_x_alert_key = []
public_x_alert_date = []
public_x_cust_id = []
public_x_size = len(public_x)
for i in range(public_x_size):
    public_x_alert_key.append(public_x['alert_key'][i])
    public_x_alert_date.append(public_x['date'][i])
    public_x_cust_id.append(information_data['cust_id'][i])
public_x_alert_date = [x for _, x in sorted(zip(public_x_alert_key, public_x_alert_date))]
public_x_alert_key.sort()



In [25]:
public_y_alert_key = []
public_y_sar_flag = []
public_y_size = len(public_y)
for i in range(public_y_size):
    public_y_alert_key.append(public_y['alert_key'][i])
    public_y_sar_flag.append(public_y['sar_flag'][i])
public_y_sar_flag = [y for _, y in sorted(zip(public_y_alert_key, public_y_sar_flag))]
public_y_alert_key.sort()

pd.DataFrame(public_y_sar_flag).to_csv('public_y.csv')

In [26]:
V_info_public = []
for i in range(public_x_size):
    risk_rank       = information_data['risk_rank'][i]
    occupation_code = information_data['occupation_code'][i]
    total_asset     = information_data['total_asset'][i]
    AGE             = information_data['AGE'][i]
    V_info_public.append([risk_rank, occupation_code, total_asset, AGE])

pd.DataFrame(V_info_public).to_csv('V_info_public.csv')

In [30]:
V_cred_public = []
month = [0, 30, 61, 91, 122, 153, 183, 214, 244, 275, 306, 334, 365]
for i in range(public_x_size):
    V_cred_public_i = []
    if public_x_cust_id[i] not in credit_data.index:
        V_cred_public_i = [0] * 8 * 13
    # search_cust = credit_data[credit_data['cust_id'] == public_x_cust_id[i]]
    else:
        search_cust = credit_data.loc[[public_x_cust_id[i]]]
        which_month = 13
        for j in range (len(month)):
            if (month[j] > public_x_alert_date[i]):
                which_month = j
                break
        for j in range (which_month-13, which_month):
            m = 0
            if (j >= 0):
                m = month[j]
            search_month = search_cust[search_cust['byymm'] == m]
            if len(search_month.to_numpy()) == 0:
                V_cred_public_i.extend([0, 0, 0, 0, 0, 0, 0, 0])
            else:
                lupay = search_month['lupay'].to_numpy()[0]
                cycam = search_month['cycam'].to_numpy()[0]
                usgam = search_month['usgam'].to_numpy()[0]
                clamt = search_month['clamt'].to_numpy()[0]
                csamt = search_month['csamt'].to_numpy()[0]
                inamt = search_month['inamt'].to_numpy()[0]
                cucsm = search_month['cucsm'].to_numpy()[0]
                cucah = search_month['cucah'].to_numpy()[0]
                V_cred_public_i.extend([lupay, cycam, usgam, clamt, csamt, inamt, cucsm, cucah])
        if (random.randint(1,100) == 56):
            print(i, public_x_size, V_cred_public_i)
    assert (len(V_cred_public_i) == 104)
    V_cred_public.append(V_cred_public_i)

pd.DataFrame(V_cred_public).to_csv('V_cred_public.csv')

127 1845 [14997.0, 165329.0, 8736.0, 0.0, 0.0, 0.0, 8457.0, 0.0, 8457.0, 165329.0, 7970.0, 0.0, 0.0, 0.0, 7355.0, 0.0, 7355.0, 165329.0, 5440.0, 0.0, 0.0, 0.0, 5440.0, 0.0, 5440.0, 165329.0, 12277.0, 0.0, 0.0, 0.0, 8407.0, 0.0, 8407.0, 165329.0, 10085.0, 0.0, 0.0, 0.0, 9600.0, 0.0, 9600.0, 165329.0, 6612.0, 0.0, 0.0, 0.0, 3901.0, 0.0, 3901.0, 165329.0, 12381.0, 0.0, 0.0, 0.0, 12381.0, 0.0, 12381.0, 165329.0, 30.0, 0.0, 0.0, 0.0, 30.0, 0.0, 30.0, 165329.0, 6139.0, 0.0, 0.0, 0.0, 6139.0, 0.0, 6139.0, 165329.0, 4453.0, 0.0, 0.0, 0.0, 4453.0, 0.0, 4453.0, 165329.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 165329.0, 1749.0, 0.0, 0.0, 0.0, 1674.0, 0.0, 1674.0, 165329.0, 73.0, 0.0, 0.0, 0.0, 73.0, 0.0]
184 1845 [1512164.0, 135240.0, -2178.0, 0.0, 0.0, 0.0, 1389814.0, 0.0, 1831.0, 135240.0, -10483.0, 0.0, 0.0, 0.0, 18910.0, 0.0, 0.0, 135240.0, -22501.0, 0.0, 0.0, 0.0, -106893.0, 0.0, 0.0, 135240.0, -157932.0, 0.0, 0.0, 0.0, -41196.0, 0.0, 128901.0, 135240.0, -56696.0, 0.0, 0.0, 0.0, 179139.0, 0.0, 3

In [31]:
V_cons_public = []
month = [0, 30, 61, 91, 122, 153, 183, 214, 244, 275, 306, 334, 365]
for i in range(public_x_size):
    V_cons_public_i = []
    if public_x_cust_id[i] not in consumption_data.index:
        V_cons_public_i = [0, 130, 47, 0] * 13
    else:
        search_cust = consumption_data.loc[[public_x_cust_id[i]]]
        country = 130
        cur_type = 47
        if len(search_cust.to_numpy()) != 0:
            country = search_cust['country'].to_numpy()[0]
            cur_type = search_cust['cur_type'].to_numpy()[0]

        which_month = 13
        for j in range (len(month)):
            if (month[j] >= public_x_alert_date[i]):
                which_month = j
                break
        for j in range (which_month-13, which_month):
            down = month[0]
            up = month[1]
            if (j > 0):
                up = month[j]
                down = month[j-1]
            search_month = search_cust[(search_cust['date'] >= down) & (search_cust['date'] < up)]

            n = len(search_month.to_numpy())
            amt = sum(search_month['amt'].to_numpy())
            V_cons_public_i.extend([n, country, cur_type, amt])
        if (random.randint(1,100) == 56):
            print(i, public_x_size, V_cons_public_i)
    assert (len(V_cons_public_i) == 52)
    V_cons_public.append(V_cons_public_i)

pd.DataFrame(V_cons_public).to_csv('V_cons_public.csv')

57 1845 [24, 130, 47, 7106.0, 24, 130, 47, 7106.0, 24, 130, 47, 7106.0, 12, 130, 47, 456.0, 11, 130, 47, 5814.0, 15, 130, 47, 8945.0, 16, 130, 47, 991.0, 21, 130, 47, 3749.0, 12, 130, 47, 5887.0, 20, 130, 47, 4044.0, 22, 130, 47, 3945.0, 7, 130, 47, 105.0, 8, 130, 47, 7057.0]
81 1845 [1, 130, 47, 144559.0, 1, 130, 47, 144559.0, 0, 130, 47, 0, 0, 130, 47, 0, 0, 130, 47, 0, 0, 130, 47, 0, 0, 130, 47, 0, 0, 130, 47, 0, 0, 130, 47, 0, 0, 130, 47, 0, 0, 130, 47, 0, 3, 130, 47, 633.0, 17, 130, 47, 3963.0]
100 1845 [1, 130, 47, 418.0, 1, 130, 47, 418.0, 1, 130, 47, 25032.0, 0, 130, 47, 0, 2, 130, 47, 124912.0, 0, 130, 47, 0, 0, 130, 47, 0, 0, 130, 47, 0, 1, 130, 47, 17390.0, 0, 130, 47, 0, 0, 130, 47, 0, 0, 130, 47, 0, 0, 130, 47, 0]
164 1845 [72, 130, 47, 621847.0, 72, 130, 47, 621847.0, 48, 130, 47, 532642.0, 53, 130, 47, 225610.0, 69, 130, 47, 233238.0, 76, 130, 47, 390277.0, 84, 130, 47, 369920.0, 75, 130, 47, 374121.0, 88, 130, 47, 303188.0, 82, 130, 47, 293085.0, 41, 130, 47, 92791.0, 3

In [32]:
V_remit_public = []
month = [0, 30, 61, 91, 122, 153, 183, 214, 244, 275, 306, 334, 365]
for i in range(public_x_size):
    V_remit_public_i = []
    if public_x_cust_id[i] not in remit_data.index:
        V_remit_public_i = [0] * 3 * 13
    else:
        search_cust = remit_data.loc[[public_x_cust_id[i]]]
        trans_no = 0
        if len(search_cust.to_numpy()) != 0:
            trans_no = search_cust['trans_no'].to_numpy()[0]
        which_month = 13
        for j in range (len(month)):
            if (month[j] >= public_x_alert_date[i]):
                which_month = j
                break

        for j in range (which_month-13, which_month):
            down = month[0]
            up = month[1]
            if (j > 0):
                up = month[j]
                down = month[j-1]
            search_day = search_cust[(search_cust['trans_date'] >= down) & (search_cust['trans_date'] < up)]
            n = len(search_day.to_numpy())
            trade_amount_usd = sum(search_day['trade_amount_usd'].to_numpy())
            V_remit_public_i.extend([n, trans_no, trade_amount_usd])
        if (random.randint(1,100) == 41):
            print(i, public_x_size, V_remit_public_i)
    assert (len(V_remit_public_i) == 39)
    V_remit_public.append(V_remit_public_i)

pd.DataFrame(V_remit_public).to_csv('V_remit_public.csv')

382 1845 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 127330.0, 1, 0, 71924.0, 15, 0, 1104802.0, 21, 0, 1890416.0, 18, 0, 1798479.0, 14, 0, 1910951.0, 18, 0, 1877500.0]
480 1845 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 188601.0, 7, 0, 306028.0, 14, 0, 953800.0]
1160 1845 [0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 1, 4, 364459.0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0]


In [33]:
V_trade_public = []
month = [0, 30, 61, 91, 122, 153, 183, 214, 244, 275, 306, 334, 365]
trading_data = pd.read_csv(data_file_path + data_file_name.format(fname = trading_file_name), index_col = 'cust_id')

for i in range(public_x_size):
    V_trade_public_i = []
    if public_x_cust_id[i] not in trading_data.index:
        V_trade_public_i = [0, 2, 0, 1.0, 0, 0, 0] * 13
    else:
        search_cust = trading_data.loc[[public_x_cust_id[i]]]
        tx_type = 2
        exchg_rate = 1.0
        lin_gui = 0
        cross_bank = 0
        ATM = 0
        if len(search_cust.to_numpy()) != 0:
            tx_type = search_cust['tx_type'].to_numpy()[0]
            exchg_rate = search_cust['exchg_rate'].to_numpy()[0]
            if ((tx_type == 1) and (search_cust['info_asset_code'].to_numpy()[0] == 12)):
                lin_gui = 1
            cross_bank = search_cust['cross_bank'].to_numpy()[0]
            ATM = search_cust['ATM'].to_numpy()[0]

        which_month = 13
        for j in range (len(month)):
            if (month[j] >= public_x_alert_date[i]):
                which_month = j
                break

        for j in range (which_month-13, which_month):
            down = month[0]
            up = month[1]
            if (j > 0):
                up = month[j]
                down = month[j-1]
            search_day = search_cust[(search_cust['tx_date'] >= down) & (search_cust['tx_date'] < up)]

            n = len(search_day.to_numpy())
            tx_amt = sum(search_day['tx_amt'].to_numpy())
            V_trade_public_i.extend([n, tx_type, tx_amt, exchg_rate, lin_gui, cross_bank, ATM])
        if (random.randint(1,100) == 98):
            print(i, public_x_size, V_trade_public_i)
    assert (len(V_trade_public_i) == 91)
    V_trade_public.append(V_trade_public_i)
  
pd.DataFrame(V_trade_public).to_csv('V_trade_public.csv')

68 1845 [3, 2, 31573.0, 1.0, 0, 1, 1, 3, 2, 31573.0, 1.0, 0, 1, 1, 3, 2, 31573.0, 1.0, 0, 1, 1, 3, 2, 17321.0, 1.0, 0, 1, 1, 4, 2, 1014636.0, 1.0, 0, 1, 1, 1, 2, 18324.0, 1.0, 0, 1, 1, 2, 2, 315577.0, 1.0, 0, 1, 1, 4, 2, 1143261.0, 1.0, 0, 1, 1, 2, 2, 1170670.0, 1.0, 0, 1, 1, 2, 2, 41737.0, 1.0, 0, 1, 1, 2, 2, 21388.0, 1.0, 0, 1, 1, 0, 2, 0, 1.0, 0, 1, 1, 4, 2, 228771.0, 1.0, 0, 1, 1]
192 1845 [4, 2, 1443340.0, 1.0, 0, 0, 0, 4, 2, 1443340.0, 1.0, 0, 0, 0, 2, 2, 48101.0, 1.0, 0, 0, 0, 7, 2, 715336.0, 1.0, 0, 0, 0, 6, 2, 518361.0, 1.0, 0, 0, 0, 19, 2, 16011001.0, 1.0, 0, 0, 0, 21, 2, 3455320.0, 1.0, 0, 0, 0, 29, 2, 4658635.0, 1.0, 0, 0, 0, 15, 2, -8209683.0, 1.0, 0, 0, 0, 13, 2, 1914044.0, 1.0, 0, 0, 0, 16, 2, 4047381.0, 1.0, 0, 0, 0, 3, 2, 32265.0, 1.0, 0, 0, 0, 14, 2, 1606749.0, 1.0, 0, 0, 0]
229 1845 [124, 2, 23001.0, 1.0, 0, 0, 1, 124, 2, 23001.0, 1.0, 0, 0, 1, 58, 2, 8391.0, 1.0, 0, 0, 1, 65, 2, 10265.0, 1.0, 0, 0, 1, 33, 2, 3557.0, 1.0, 0, 0, 1, 52, 2, 6534.0, 1.0, 0, 0, 1, 32, 2, 